In [6]:
import pandas as pd

# Path to your Parquet gzip file
file_path_11 = '../../1_Data/CLEANED/interventions_dataset.parquet'

# Read the Parquet file into a pandas DataFrame
interventions_dataset = pd.read_parquet(file_path_11, engine='pyarrow')

In [2]:
# Make dummies for categorical variables
df_dummies = pd.get_dummies(interventions_dataset, columns=['vector_type', 'eventtype_firstcall', 'eventlevel_firstcall', 'eventtype_trip', 'eventlevel_trip', 
                                                            'province_intervention', 'number_of_transported_persons', 'abandon_reason'])

# Assuming df_interventions1 is your DataFrame
correlation_matrix = df_dummies.corr()

# Filter the correlation matrix to include only correlations higher than 0.15 but lower than 1
filtered_correlation_matrix = correlation_matrix[
    (correlation_matrix > 0.2) & (correlation_matrix < 1)
]

# Create a list to store the correlation pairs and values
correlation_list = []

# Iterate over rows and columns of the filtered correlation matrix
for col in filtered_correlation_matrix.columns:
    for index, value in filtered_correlation_matrix[col].items():
        # Check if the correlation value is not NaN and append to the list
        if not pd.isnull(value):
            if value > 0.2 and value < 1:
                correlation_list.append((col, index, value))

# Sort the correlation list based on correlation values in descending order
correlation_list.sort(key=lambda x: x[2], reverse=True)

# Print the ranked correlations
for pair in correlation_list:
    print(f"{pair[0]} - {pair[1]}: {pair[2]}")

/var/folders/t3/ph8rkyks35s2z67jf7rffj9w0000gn/T/ipykernel_24928/1956453753.py:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df_dummies.corr()


t0_Year - t7_Year: 0.9995910128866842
t7_Year - t0_Year: 0.9995910128866842
t0_Month - t7_Month: 0.9983321242305413
t7_Month - t0_Month: 0.9983321242305413
longitude_permanence - longitude_intervention: 0.9950230925269198
longitude_intervention - longitude_permanence: 0.9950230925269198
latitude_permanence - latitude_intervention: 0.9918363631837368
latitude_intervention - latitude_permanence: 0.9918363631837368
eventtype_firstcall_P035 - Convulsions child < 7 years old - eventtype_trip_P035 - Convulsions child < 7 years old: 0.9898004774724595
eventtype_trip_P035 - Convulsions child < 7 years old - eventtype_firstcall_P035 - Convulsions child < 7 years old: 0.9898004774724595
intervention_time_(t1reported) - unavailable_time: 0.9845706710350748
unavailable_time - intervention_time_(t1reported): 0.9845706710350748
waiting_time - unavailable_time: 0.981745200818155
unavailable_time - waiting_time: 0.981745200818155
eventtype_firstcall_P090 - Test MED - eventtype_trip_P090 - Test MED: 0.

In [3]:
import pandas as pd

# Assuming df_interventions1 is your DataFrame
selected_columns = ['vector_type', 'eventtype_trip']

# Selecting the relevant columns
df_selected = interventions_dataset[selected_columns]

# Encoding categorical variables using one-hot encoding
df_encoded = pd.get_dummies(df_selected)

# Calculating the correlation matrix
correlation_matrix = df_encoded.corr()

# Selecting the correlation values for vector_type_MUG and the dummy variables of eventtype_trip
mug_correlation = correlation_matrix['vector_type_MUG'].loc[df_encoded.columns[df_encoded.columns.str.startswith('eventtype_trip')]]

# Sorting the correlations in descending order
top_5_correlations = mug_correlation.sort_values(ascending=False).head(5)

# Create a DataFrame to display the results
top_5_correlations_df = pd.DataFrame({'Eventtype_Trip': top_5_correlations.index, 'Correlation': top_5_correlations.values})

# Display the DataFrame
print(top_5_correlations_df)


                                  Eventtype_Trip  Correlation
0               eventtype_trip_P011 - Chest pain     0.144967
1    eventtype_trip_P019 - Unconscious - syncope     0.121773
2           eventtype_trip_P003 - Cardiac arrest     0.104187
3   eventtype_trip_P015 - Epilepsy - convulsions     0.088181
4  eventtype_trip_P099 - Interhospital transport     0.062108


In [4]:
import pandas as pd

# Assuming df_interventions1 is your DataFrame
selected_columns = ['vector_type', 'eventtype_trip']

# Selecting the relevant columns
df_selected = interventions_dataset[selected_columns]

# Encoding categorical variables using one-hot encoding
df_encoded = pd.get_dummies(df_selected)

# Calculating the correlation matrix
correlation_matrix = df_encoded.corr()

# Selecting the correlation values for vector_type_MUG and the dummy variables of eventtype_trip
mug_correlation = correlation_matrix['vector_type_Brandziekenwagen'].loc[df_encoded.columns[df_encoded.columns.str.startswith('eventtype_trip')]]

# Sorting the correlations in descending order
top_5_correlations = mug_correlation.sort_values(ascending=False).head(5)

# Create a DataFrame to display the results
top_5_correlations_df = pd.DataFrame({'Eventtype_Trip': top_5_correlations.index, 'Correlation': top_5_correlations.values})

# Display the DataFrame
print(top_5_correlations_df)

                                      Eventtype_Trip  Correlation
0            eventtype_trip_FI (1.3.0) fire building     0.416905
1                eventtype_trip_HG (2.1.1) gas odour     0.315338
2                 eventtype_trip_HG (2.1.2) gas leak     0.242795
3          eventtype_trip_TI (3.3.2) CO intoxication     0.232573
4  eventtype_trip_TI (3.3.3) rescue from danger t...     0.112504


In [7]:
import pandas as pd

# Filter dataset to include only the relevant columns
columns_of_interest = ['Month', 'Weekday', 'EventType Trip']
filtered_df = interventions_dataset[columns_of_interest]

# One-hot encode EventType Trip
filtered_df = pd.get_dummies(filtered_df, columns=['EventType Trip'])

# Calculate correlation matrix
correlation_matrix = filtered_df.corr()

# Extract top 20 highest correlations
correlation_pairs = correlation_matrix.unstack().sort_values(ascending=False)
top_correlations = correlation_pairs[correlation_pairs < 1].head(20)

print("Top 20 highest correlations between month, and weekday vs event type:")
print(top_correlations)

KeyError: "None of [Index(['Month', 'Weekday', 'EventType Trip'], dtype='object')] are in the [columns]"

In [8]:
import pandas as pd

# Create dummy variables for event types
dummy_variables = pd.get_dummies(interventions_dataset['eventtype_trip'], prefix='event_type')

# Concatenate the original DataFrame with the dummy variables
interventions_dataset = pd.concat([interventions_dataset, dummy_variables], axis=1)

# Create a list of column names representing the dummy variables for event types
list_of_dummy_variables = dummy_variables.columns.tolist()

# Create a DataFrame containing only the relevant columns
relevant_columns = interventions_dataset[['t0_Hour', 't0_Day', 't0_Month'] + list_of_dummy_variables]

# Calculate the correlation matrix
correlation_matrix = relevant_columns.corr()

# Extract the correlations of t0_Hour, t0_Day, and t0_Month with the dummy variables for event types
correlations = correlation_matrix.loc[['t0_Hour', 't0_Day', 't0_Month'], list_of_dummy_variables]

# Get the absolute values of correlations
absolute_correlations = correlations.abs()

# Get the top 20 correlations
top_20_correlations = absolute_correlations.stack().nlargest(20)

print("Top 20 correlations between t0_Hour, t0_Day, t0_Month, and event types:")
print(top_20_correlations)

In [6]:
import numpy as np

# Select the columns of interest
categorical_columns = ['vector_type', 'eventlevel_trip']

# Convert categorical variables into dummy variables
dummy_variables = pd.get_dummies(interventions_dataset[categorical_columns])

# Compute the correlation matrix
correlation_matrix = dummy_variables.corr()

# Filter out one of the pairs from the upper triangular part of the correlation matrix
correlation_matrix = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Extract correlation values for the dummy variables of interest
correlation_values = correlation_matrix.unstack().sort_values(ascending=False)

# Remove correlation values of 1 (correlation of a variable with itself) and convert to DataFrame
correlation_df = pd.DataFrame(correlation_values[correlation_values != 1], columns=['Correlation'])

# Get the top 5 highest correlations
top_20_correlations = correlation_df.head(20)

print(top_20_correlations)

                                                       Correlation
eventlevel_trip_N5  vector_type_Ambulance                 0.493346
eventlevel_trip_N1  vector_type_MUG                       0.374699
eventlevel_trip_N4  vector_type_PIT                       0.322350
eventlevel_trip_N2  vector_type_MUG                       0.272858
eventlevel_trip_N3  vector_type_PIT                       0.231451
                    vector_type_MUG                       0.128317
eventlevel_trip_N0  vector_type_MUG                       0.063472
eventlevel_trip_N3  vector_type_Ambulance Exceptional     0.048790
eventlevel_trip_N6  vector_type_Ambulance                 0.015840
eventlevel_trip_N1  vector_type_MUG Event                 0.014887
eventlevel_trip_N5  vector_type_Ambulance Event           0.009829
eventlevel_trip_N6  vector_type_MUG Event                 0.006473
                    vector_type_Ambulance Event           0.006177
eventlevel_trip_N8  vector_type_Ambulance Event           0.00